In this tutorial we will go through the basic training of an object detection model with your own annotated images.

The model you will use is a Tiny YOLO v4 from the DarkNet[https://github.com/AlexeyAB/darknet]. 
Will run through the following steps:


*   Install the libraries
*   Train the model on the new images
*   Run inference on a few images to see what the model can detect
*   Convert the model to OpenVINO Intermediate Representation
*   Run inference again to see if the IR model works similarly as well

# Install Libraries
# Press Shift+Enter to run cells
Some cells are commented out (all text is green), so they will not run the code inside.
To uncomment code inside a cell, select all the code then press 'Ctrl' + '/'

In [ ]:
# Install Requirements - Cmake, Cuda, Cudnn, opencv
# Installing cmake
# First download from the website, https://cmake.org/download/, download the source code package - cmake-3.11.1.tar.gz 
# Decompress downloaded cmake zip file using
# %%capture
!tar -zvxf cmake-3.11.1.tar.gz
%cd cmake-3.11.1
!./bootstrap
!./configure --prefix=/home/xxx/xxx #---The directory under your user name
!make
!make install
Then add the /home/xxx/xxx/bin directory to the environment variable, and save the source
!cmake -version

# Setting cuda variable if cuda is already present
%env CUDA_HOME=/usr/local/cuda
%env LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/usr/local/cuda/lib64:/usr/local/cuda/extras/CUPTI/lib64
%env PATH=$PATH:$CUDA_HOME/bin

# Installing cudnn - https://docs.nvidia.com/deeplearning/cudnn/install-guide/index.html#download
!tar -xvf cudnn-11.4-linux-x64-v8.2.4.15.tgz
!sudo cp cuda/include/cudnn*.h /usr/local/cuda/include
!sudo cp -P cuda/lib64/libcudnn* /usr/local/cuda/lib64
!sudo chmod a+r /usr/local/cuda/include/cudnn*.h /usr/local/cuda/lib64/libcudnn*
!sudo /bin/bash -c 'echo "/usr/local/lib" > /etc/ld.so.conf.d/opencv.conf'
!sudo ldconfig
# Installing opencv - https://linuxize.com/post/how-to-install-opencv-on-ubuntu-18-04/


### Install ulc-malaria-scope

In [ ]:
!git clone https://github.com/czbiohub/ulc-malaria-scope

### Install Darknet

In [ ]:
!git clone https://github.com/AlexeyAB/darknet
!cd darknet
!mkdir build_release
!cd build_release
!cmake ..
!cmake --build . --target install --parallel 8

### Install luminoth

In [ ]:
!pip install -e "git+https://github.com/czbiohub/luminoth.git#egg=master"

In [ ]:
!pip install click==7.1.2

In [ ]:
%env LC_ALL=C.UTF-8

In [ ]:
%env LANG=C.UTF-8

### Test if luminoth works

In [ ]:
!apt-get -y update && apt-get -y install -y libgl1

In [ ]:
!lumi --help

### Use lumi to split training and validation data

Own data should be split into 80% training (train folder), 20% for validation (test folder). The train and test folder should contain images and associated annotations in csv.

Have extra images for final model testing (final_test folder). These need not be annotated.

In [ ]:
# Optional mosaicing leica data given tiles in folders of classes namely healthy, ring, schizont, troph and obtaining a bb_labels.csv using
%cd ulc-malaria-scope
!python3 mosaic_to_leica_format.py
# Use lumi to split to get train, val images, csv file
!lumi split-train-val /data/bioengineering_data/leica_intel_ssd_data/bb_labels.csv --output_dir /data/bioengineering_data/leica_intel_yolo_data/ --percentage 0.8 --random_seed 42 --input_image_format .png

# Here if the csv file contains the image_id column it might have to be replaced to say filename using below command

!sed -i '1s/.*/filename,xmin,xmax,ymin,ymax,label/' //data/bioengineering_data/leica_intel_ssd_data/lumi_csv/train.csv
!sed -i '1s/.*/filename,xmin,xmax,ymin,ymax,label/' //data/bioengineering_data/leica_intel_ssd_data/lumi_csv/val.csv

Convert lumi single csv file to yolo required text files each for a image containing only bounding boxes for that image, labels.txt contains 4 lines with 4 labels healthy ring schizont troph

In [ ]:
!python3 intel_detection_utils/csv_to_text.py --input /mnt/data_lg/pranathi/bioengineering_data/leica_intel_yolo_data/ --image_format .jpg --labels labels.txt -r "800X600"



Move images and text files created in the above step to /home/vemuri/code/darknet/build/darknet/x64/data/obj/
Create a file named obj.names in the same folder and list the labels healthy ring schizont troph each in a separate line 
/home/vemuri/code/darknet/build/darknet/x64/data/obj.names

Tune parameters for your data in darknet using this link[https://github.com/AlexeyAB/darknet#how-to-train-to-detect-your-custom-objects]

In [1]:
# cat /home/vemuri/code/darknet/build/darknet/x64/data/obj.data
# classes = 4
# train  = /home/vemuri/code/darknet/build/darknet/x64/data/train.txt
# valid  = /home/vemuri/code/darknet/build/darknet/x64/data/test.txt
# names = /home/vemuri/code/darknet/build/darknet/x64/data/obj.names
# backup = /mnt/data_lg/pranathi/backup/

## Train using Darknet

In [ ]:
%cd darknet
!./darknet detector train /home/vemuri/code/darknet/build/darknet/x64/data/obj.data /home/vemuri/code/ulc-malaria-scope/yolov4-tiny-obj.cfg /home/vemuri/code/darknet/build/darknet/x64/yolov4-tiny.conv.29 -map

In [ ]:
# Run inference on test dataset, visualize results, and generate confusion matrix

In [ ]:
%cd ulc-malaria-scope
!python3 intel_detection_utils/make_images_text_file.py --input /mnt/data_lg/pranathi/bioengineering_data/leica_intel_yolo_data/val/ --image_format .jpg --output_text_path val_leica.txt
%cd darknet
!./darknet detector test /home/vemuri/code/darknet/build/darknet/x64/data/obj.data yolov4-tiny-obj.cfg /mnt/data_lg/pranathi/backup/yolov4-tiny-obj_best.weights -thresh 0.4 -dont_show -save_labels < /home/vemuri/val_leica.txt
# Convert text files containing annotations for each image to a csv file
!python3 text_to_csv.py --input /mnt/data_lg/pranathi/bioengineering_data/leica_intel_yolo_data/val/ --image_format .jpg --labels labels.txt --images_text_path /home/vemuri/val_leica.txt --resolution "800X600"
# Overlay bounding boxes from the predicted csv into images 
lumi overlay-bbs --im_dir ./val --csv_path ./val/bb_labels.csv  --output_dir overlaid_mosaic_yolo_leica --input_image_format .jpg
# Get confusion matrix
lumi confusion-matrix --groundtruth_csv /Users/pranathi.vemuri/czbiohub/ulc-malaria-scope/val.csv --predicted_csv /Users/pranathi.vemuri/czbiohub/ulc-malaria-scope/val/bb_labels.csv --output_txt output_yolo_val_leica_feb2022.txt  --num_cpus 1 --classes_json /Users/pranathi.vemuri/czbiohub/ulc-malaria-scope/models/classes.json--output_fig output_yolo_val_leica_feb2022.png

In [ ]:
#model dir check for the trained model
!ls /mnt/data_lg/pranathi/backup/

## Export a Trained Inference Graph to .pb
Once your training job is complete, you need to extract the newly trained inference graph, which will be later used to perform the object detection. This can be done as follows:

In [ ]:
%cd /home/vemuri/code/ulc-malaria-scope/ulc_mm_package/OpenVINO-YOLOV4/

In [ ]:
!python3 convert_weights_pb.py --class_names /home/vemuri/code/darknet/build/darknet/x64/data/obj.names --weights_file /mnt/data_lg/pranathi/backup/yolov4-tiny-obj_best.weights --data_format NHWC --tiny

In [ ]:
# .pb file will be created in the same directory check
!ls ulc-malaria-scope/ulc_mm_package/OpenVINO-YOLOV4/

# Convert TF model to OpenVINO 2021.4 Intermediate Representation (IR)
 This can be used to run inference on OpenVINO.

## First, we install OpenVINO 2021.4


Installing openvivo toolkit library for mac using[https://www.intel.com/content/www/us/en/developer/tools/openvino-toolkit-download.html?operatingsystem=mac&distributions=webdownload&version=2021%204.2%20LTS%20(latest)&options=offline]
Then installing dmg file and following installation instructions it redirects to at the end of installation finish notice

[Optional] OpenVino install check, generally not needed

In [ ]:
# !source /opt/intel/openvino/bin/setupvars.sh && \
#     /opt/intel/openvino/deployment_tools/demo/demo_squeezenet_download_convert_run.sh

## Convert TF model to Open Vino Intermediate Representation
If using own model, please change to your desired name for output directory --output_dir "choose name"

In [ ]:
%cd "/home/vemuri/code/ulc-malaria-scope/ulc_mm_package/OpenVINO-YOLOV4/"
!source /opt/intel/openvino_2021/bin/setupvars.sh && \
    mo.py --input_model frozen_darknet_yolov4_model.pb --transformations_config yolo_v4_tiny.json --batch 1 --reverse_input_channels

## Running inference using IR model on CPU/Intel Neural Compute stick (.xml, .bin file)

Set device flag in below commands to CPU or MYRIAD based on if you want to infer using CPU or Neural compute stick

In [ ]:
# Synchronus
%cd /home/vemuri/code/ulc-malaria-scope/ulc_mm_package/OpenVINO-YOLOV4/pythondemo/2021.3/
!python3.9 sync_detection_yolo.py -i /Users/pranathi.vemuri/Documents/2022-01-21-155910-229209.avi -d MYRIAD -m ../OpenVINO-YOLOV4/frozen_darknet_yolov4_model.xml --o op_basler -f .avi


In [ ]:
# Asynchronus
%cd /home/vemuri/code/ulc-malaria-scope/ulc_mm_package/OpenVINO-YOLOV4/pythondemo/2021.3/
!python3.9 object_detection_demo_yolov3_async.py -i /Users/pranathi.vemuri/czbiohub/ulc-malaria-scope/val/out.mp4 -m frozen_darknet_yolov4_model.xml  -d MYRIAD


# Convert to tflite by first converting to SavedModel Pb and then tflite 

## Detect using yolo tflite model above

In [ ]:
%cd ulc-malaria-scope/ulc-mm-package/detection

In [ ]:
!python detectvideo.py \
    --framework tflite \
    --weights checkpoints/yolov4-416.tflite --tiny \
    --video /Users/pranathi.vemuri/czbiohub/all_results/val/out_val.mp4
    --output /Users/pranathi.vemuri/czbiohub/all_results/val/op_tflite


# Convert tflite to edgetpu tflite model
Convert using edgetpu_compiler - https://colab.research.google.com/github/google-coral/tutorials/blob/master/compile_for_edgetpu.ipynb or from step 2 in https://coral.ai/docs/edgetpu/retrain-detection/#compile-the-model-for-the-edge-tpu

Currently errors out for the model obtained above

! edgetpu_compiler $TFLITE_FILE
Edge TPU Compiler version 16.0.384591198
Started a compilation timeout timer of 180 seconds.
ERROR: Attempting to use a delegate that only supports static-sized tensors with a graph that has dynamic-sized tensors.
Compilation failed: Model failed in Tflite interpreter. Please ensure model can be loaded/run in Tflite interpreter.
Compilation child process completed within timeout period.
Compilation failed! 

# Confusion matrix on validation data for tflite converted and IR model

In [ ]:
!lumi confusion-matrix --groundtruth_csv /Users/pranathi.vemuri/czbiohub/all_results/val/bb_labels.csv --predicted_csv /Users/pranathi.vemuri/czbiohub/all_results/op_basler/bb_labels.csv --output_txt output_yolo_tflite  --num_cpus 1 --classes_json /Users/pranathi.vemuri/czbiohub/ulc-malaria-scope/ssd-models/classes.json --output_fig output_yolo_tflite.png

!lumi confusion-matrix --groundtruth_csv /Users/pranathi.vemuri/czbiohub/all_results/val/bb_labels.csv --predicted_csv /Users/pranathi.vemuri/czbiohub/all_results/val/op_tflite/bb_labels.csv --output_txt output_yolo_ir.txt  --num_cpus 1 --classes_json /Users/pranathi.vemuri/czbiohub/ulc-malaria-scope/ssd-models/classes.json --output_fig output_yolo_ir.png

# Titration experiments

In [ ]:
!python3.9 /Users/pranathi.vemuri/czbiohub/ulc-malaria-scope/ulc_mm_package/OpenVINO-YOLOV4/pythondemo/2021.3/sync_detection_yolo.py -i /Volumes/flexo/MicroscopyData/Bioengineering/Label-Free\ Malaria\ Paper/Processed\ Data/Leica/Titration/Point\ 1/405nm\ 40x_aligned  -d MYRIAD -m /Users/pranathi.vemuri/czbiohub/ulc-malaria-scope/yolov4-models/frozen_darknet_yolov4_model.xml --o "op_Point1" -f .tif --labels /Users/pranathi.vemuri/czbiohub/ulc-malaria-scope/yolov4-models/obj.names

!python3.9 /Users/pranathi.vemuri/czbiohub/ulc-malaria-scope/ulc_mm_package/OpenVINO-YOLOV4/pythondemo/2021.3/sync_detection_yolo.py -i /Volumes/flexo/MicroscopyData/Bioengineering/Label-Free\ Malaria\ Paper/Processed\ Data/Leica/Titration/Point\ 2/405nm\ 40x_aligned  -d MYRIAD -m /Users/pranathi.vemuri/czbiohub/ulc-malaria-scope/yolov4-models/frozen_darknet_yolov4_model.xml --o "op_Point2" -f .tif --labels /Users/pranathi.vemuri/czbiohub/ulc-malaria-scope/yolov4-models/obj.names

!python3.9 /Users/pranathi.vemuri/czbiohub/ulc-malaria-scope/ulc_mm_package/OpenVINO-YOLOV4/pythondemo/2021.3/sync_detection_yolo.py -i /Volumes/flexo/MicroscopyData/Bioengineering/Label-Free\ Malaria\ Paper/Processed\ Data/Leica/Titration/Point\ 3/405nm\ 40x_aligned  -d MYRIAD -m /Users/pranathi.vemuri/czbiohub/ulc-malaria-scope/yolov4-models/frozen_darknet_yolov4_model.xml --o "op_Point3" -f .tif --labels /Users/pranathi.vemuri/czbiohub/ulc-malaria-scope/yolov4-models/obj.names

!python3.9 /Users/pranathi.vemuri/czbiohub/ulc-malaria-scope/ulc_mm_package/OpenVINO-YOLOV4/pythondemo/2021.3/sync_detection_yolo.py -i /Volumes/flexo/MicroscopyData/Bioengineering/Label-Free\ Malaria\ Paper/Processed\ Data/Leica/Titration/Point\ 4/405nm\ 40x_aligned  -d MYRIAD -m /Users/pranathi.vemuri/czbiohub/ulc-malaria-scope/yolov4-models/frozen_darknet_yolov4_model.xml --o "op_Point4" -f .tif --labels /Users/pranathi.vemuri/czbiohub/ulc-malaria-scope/yolov4-models/obj.names

!python3.9 /Users/pranathi.vemuri/czbiohub/ulc-malaria-scope/ulc_mm_package/OpenVINO-YOLOV4/pythondemo/2021.3/sync_detection_yolo.py -i /Volumes/flexo/MicroscopyData/Bioengineering/Label-Free\ Malaria\ Paper/Processed\ Data/Leica/Titration/Point\ 5/405nm\ 40x_aligned  -d MYRIAD -m /Users/pranathi.vemuri/czbiohub/ulc-malaria-scope/yolov4-models/frozen_darknet_yolov4_model.xml --o "op_Point5" -f .tif --labels /Users/pranathi.vemuri/czbiohub/ulc-malaria-scope/yolov4-models/obj.names

!python3.9 /Users/pranathi.vemuri/czbiohub/ulc-malaria-scope/ulc_mm_package/OpenVINO-YOLOV4/pythondemo/2021.3/sync_detection_yolo.py -i /Volumes/flexo/MicroscopyData/Bioengineering/Label-Free\ Malaria\ Paper/Processed\ Data/Leica/Titration/Point\ 6/405nm\ 40x_aligned  -d MYRIAD -m /Users/pranathi.vemuri/czbiohub/ulc-malaria-scope/yolov4-models/frozen_darknet_yolov4_model.xml --o "op_Point6" -f .tif --labels /Users/pranathi.vemuri/czbiohub/ulc-malaria-scope/yolov4-models/obj.names

!python3.9 /Users/pranathi.vemuri/czbiohub/ulc-malaria-scope/ulc_mm_package/OpenVINO-YOLOV4/pythondemo/2021.3/sync_detection_yolo.py -i /Volumes/flexo/MicroscopyData/Bioengineering/Label-Free\ Malaria\ Paper/Processed\ Data/Leica/Titration/Point\ 7/405nm\ 40x_aligned  -d MYRIAD -m /Users/pranathi.vemuri/czbiohub/ulc-malaria-scope/yolov4-models/frozen_darknet_yolov4_model.xml --o "op_Point7" -f .tif --labels /Users/pranathi.vemuri/czbiohub/ulc-malaria-scope/yolov4-models/obj.names

!python3.9 /Users/pranathi.vemuri/czbiohub/ulc-malaria-scope/ulc_mm_package/OpenVINO-YOLOV4/pythondemo/2021.3/sync_detection_yolo.py -i /Volumes/flexo/MicroscopyData/Bioengineering/Label-Free\ Malaria\ Paper/Processed\ Data/Leica/Titration/Point\ 8/405nm\ 40x_aligned  -d MYRIAD -m /Users/pranathi.vemuri/czbiohub/ulc-malaria-scope/yolov4-models/frozen_darknet_yolov4_model.xml --o "op_Point8" -f .tif --labels /Users/pranathi.vemuri/czbiohub/ulc-malaria-scope/yolov4-models/obj.names

!python3.9 /Users/pranathi.vemuri/czbiohub/ulc-malaria-scope/ulc_mm_package/OpenVINO-YOLOV4/pythondemo/2021.3/sync_detection_yolo.py -i /Volumes/flexo/MicroscopyData/Bioengineering/Label-Free\ Malaria\ Paper/Processed\ Data/Leica/Titration/Point\ 9/405nm\ 40x_aligned  -d MYRIAD -m /Users/pranathi.vemuri/czbiohub/ulc-malaria-scope/yolov4-models/frozen_darknet_yolov4_model.xml --o "op_Point9" -f .tif --labels /Users/pranathi.vemuri/czbiohub/ulc-malaria-scope/yolov4-models/obj.names

!python3.9 /Users/pranathi.vemuri/czbiohub/ulc-malaria-scope/ulc_mm_package/OpenVINO-YOLOV4/pythondemo/2021.3/sync_detection_yolo.py -i /Volumes/flexo/MicroscopyData/Bioengineering/Label-Free\ Malaria\ Paper/Processed\ Data/Leica/Titration/Point\ 10/405nm\ 40x_aligned  -d MYRIAD -m /Users/pranathi.vemuri/czbiohub/ulc-malaria-scope/yolov4-models/frozen_darknet_yolov4_model.xml --o "op_Point10" -f .tif --labels /Users/pranathi.vemuri/czbiohub/ulc-malaria-scope/yolov4-models/obj.names